# Stress Test

The idea of this code is to see how the production Endpoint will behave when a **bunch** of requests arrive it.
Let's simulate several users doing predictions at the same time

In [ ]:
import threading
import boto3
import json
import numpy as np
import time
import math

from multiprocessing.pool import ThreadPool
from sklearn import datasets

In [ ]:
sm = boto3.client("sagemaker-runtime")
codepipeline = boto3.client('codepipeline')

endpoint_name='mlops-iris-model-prd-%s'
pipeline_name = 'mlops-iris-model'

iris = datasets.load_iris()
dataset = np.insert(iris.data, 0, iris.target,axis=1)

In [ ]:
def get_env_jobid(env):
    response = codepipeline.get_pipeline_state( name=pipeline_name )
    for stage in response['stageStates']:
        if stage['stageName'] == 'Deploy%s' % env.capitalize():
            for action in stage['actionStates']:
                if action['actionName'] == 'DeployModel%s' % env.capitalize():
                    return stage['latestExecution']['pipelineExecutionId']

In [ ]:
def predict(payload):
    payload = payload
    X = [ payload[1:] ]
    y = payload[0]
    response = []
    elapsed_time = time.time()
    resp = sm.invoke_endpoint(
        EndpointName=endpoint_name % env,
        Body=json.dumps({"algorithm": "logistic", "payload": X })
    )
    elapsed_time = time.time() - elapsed_time
    resp = json.loads(resp['Body'].read())
    response.append((resp['iris_id'][0] == y, elapsed_time))
    
    elapsed_time = time.time()
    resp = sm.invoke_endpoint(
        EndpointName=endpoint_name % env,
        Body=json.dumps({"algorithm": "random_forest", "payload": X })
    )
    elapsed_time = time.time() - elapsed_time
    resp = json.loads(resp['Body'].read())
    response.append((resp['iris_id'][0] == y, elapsed_time))
    
    return response

In [ ]:
def run_test(max_threads, max_requests):
    num_batches = math.ceil(max_requests / len(dataset))
    requests = []
    for i in range(num_batches):
        batch = dataset.copy()
        np.random.shuffle(batch)
        requests += batch.tolist()
    len(requests)

    pool = ThreadPool(max_threads)
    result = pool.map(predict, requests)
    pool.close()
    pool.join()
    
    correct_logistic=0
    correct_random_forest=0
    elapsedtime_logistic=0
    elapsedtime_random_forest=0
    for i in result:
        correct_logistic += i[0][0]
        correct_random_forest += i[1][0]

        elapsedtime_logistic += i[0][1]
        elapsedtime_random_forest += i[1][1]
    print("Score logistic: {}".format(correct_logistic/len(result)))
    print("Score random forest: {}".format(correct_random_forest/len(result)))

    print("Elapsed time logistic: {}s".format(elapsedtime_logistic))
    print("Elapsed time random forest: {}s".format(elapsedtime_random_forest))

In [ ]:
env=get_env_jobid('prd')

In [ ]:
%%time
print("Starting test 1")
run_test(10, 1000)

> While this test is running, go to the **AWS Console** -> **Sagemaker**, then click on the **Endpoint** and then click on the **CloudWatch** monitoring logs to see the Endpoint Behavior

In [ ]:
%%time
print("Starting test 2")
run_test(100, 10000)

In [ ]:
%%time
print("Starting test 3")
run_test(150, 100000)

## Well done!